# **LAB ONE: NMR SIGNAL**

Brief description of lab

#### Learning Objectives:
- Learn about magnetic fields in NMR
- Learn what NMR is
- Learn how to excite an NMR signal in a sample
- Learn how to measure the NMR signal
- Learn what causes NMR signal decay
- Learn how to optimise amplitude and duration
- Learn what an NMR spin echo is

#### Assumed Knowledge:
- LC Resonance 
- Solenoid Coils
- Fourier Transform
- Angular Momentum 
- Complex Numbers


-------------------------------------------------------------------------------------------------------------------------------------------------------
#### **Setup Task:**

To initialise this notebook, edit the cell above to set `LAB_USER_NAME` to your name, then click **Run->Run All Cells** in the top menu bar  




In [1]:
LAB_USER_NAME = 'Spider'

In [2]:
import panel as pn
pn.extension()
import sys
import os
sys.path.append('/home/notebooks/dashboards-inline')
LAB_DIR = os.path.join('/home/data/', LAB_USER_NAME)
os.makedirs(LAB_DIR, exist_ok=True)
print('Data will be saved to', LAB_DIR)

Data will be saved to /home/data/Spider


-------------------------------------------------------------------------------------------------------------------------------------------------------

## 1. Magnetic Fields (B0 and B1)

- B0 show the parts of the machine that controls this field and discuss the direction of field (compare to standard MRI?)
- 
-
-
-
- Do we want to discuss the location of the tuning capacitors and stuff here or later on when we get them to tune things?????

add larmor equation - how larmor freq relates to b0

The $B_1$ field is generated by an RF coil in parallel with a capacitor which functions as a simple RLC resonator circuit. The resonant frequency of an RLC circuit is as follows:

$$f_0 = \frac{1}{2\pi\sqrt{LC}}$$

Adjusting the *Tune* capacitor on the probe changes the value of $C$. The RLC resonator circuit also needs to be impedance matched to the RF transmit amplifier and receive preamplifier, which have a characteristic impedance of 50 Ohms. This is accomplished with a *Match* capacitor.

The Wobble tool plots the $S_{11}$ curve of the RF coil, which indicates the tune and match of the RF coil. An $S_{11}$ plots the reflected signal of the the coil as a function of frequency. The position of the dip in the $S_{11}$ curve is mostly controlled by the *Tune* capacitor, while the depth of the dip is mostly controlled by the *Match* capacitor, though there is some overlap in effect. Power will be transfered to the coil most efficiently when the amount of reflected power measured by $S_{11}$ is minimised. 

**NO S11 - Explain!**

Diagrams:
- render of the magnet: show b0 and b1 - perpendicular
- cap circuit

<center><img src="Images/CapCircuit.png" width="500"></center>

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 1.1: Measure Magnetic Field Strength**
> 
> - Hall probe and compass to find the direction and strength of the magnetic field
> - Record the field strength
> 
> -------------------------------------------------------------------------------------------------------------------------------------------------------

## 2. Precession and Tuning to Larmor Frequency

Some explaination of precession and NMR

Equation for the Larmor frequency

Diagram: add angular momentum


<center><img src="Images/PrecessionDiagram.png" width="200"></center>

>--- 
> #### **Task 2.1: Calculate Larmor Frequency**
>
> The $^1\mathrm{H}$ nucleus has a gyromagnetic ratio of 42.576 MHz/T and a system field strength of about 0.3425 T.  
>
> Calculate the desired frequency of the system and enter your calculated value in Hz below (scientific notation will work, e.g. "5.5e6"):
>
>--- 

In [3]:
frequency = pn.widgets.FloatInput(name='Frequency (Hz)', start=0, step=1000)
frequency

FloatInput(name='Frequency (Hz)', start=0, step=1000)



**TO DO: Reintroduce the wobble tool and what it does.** 


> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 2.2: Tune the Probe**
> - Remove the black plate on top of the ilumr by rotating it anticlockwise.
> - Run the **Wobble** tool below in loop mode. 
> - While the **Wobble** tool is running, adjust the variable capacitors with the provided screwdriver to tune and match the RF coil to your calculated Larmor frequency (Note: the dotted line on the plot corresponds to the desired frequency). **TO DO: show picture of caps. add description of how to know when the probe is tuned enough**
> - Include the dB measurement of your coil tune - this will require a simple calculation. *we need to reword this slightly*
> - Load a sample into the ilumr and observe how "loading" a coil will affect both the tune and match of the coil.
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [4]:
from Wobble import WobbleApp # from dashboards-inline directory that was added to sys.path
wobble = WobbleApp(
    override_pars=dict(
        f=frequency, # use value of fequency from the user input
        f_bw=2e6 # fix bandwidth at 2 MHz
    )
)
wobble.main()

Column(sizing_mode='stretch_both')
    [0] Row(sizing_mode='stretch_both')
        [0] Bokeh(Figure)
    [1] Row(sizing_mode='stretch_width')
        [0] Button(align='end', button_type='success', name='Run', sizing_mode='stretch_width')
        [1] Button(align='end', button_type='success', name='Run Loop', sizing_mode='stretch_width')
        [2] Button(align='end', button_type='danger', name='Abort', sizing_mode='stretch_width')
        [3] TextInput(align='end', disabled=True, sizing_mode='stretch_width', value='Idle')

TO DO: ADD CHART TITLES

NOTE: If using **Run Loop**, click abort before continuing


> ---
> #### **Task 2.3: Find NMR Signal** 
> - Use the depth gauge to align the **shim sample** with the 10mm point.
> - Load the **shim sample** into the ilumr.
> - Input your calculated frequency and a starting RF amplitude guess of $0.5$ (half of maximum) into the boxes below.
> - Run the **FID** tool below. 
> ---


In [5]:
rf_amp = pn.widgets.FloatInput(name='RF Amplitude (fraction of maximum)', start=0, end=1, step=0.1, value=0.5)
# Also display frequency again, as FID uses it
pn.Row(frequency, rf_amp)

Row
    [0] FloatInput(name='Frequency (Hz)', start=0, step=1000)
    [1] FloatInput(end=1, name='RF Amplitude (..., start=0, value=0.5)

In [6]:
from FID import FIDApp # from dashboards-inline directory that was added to sys.path

# setup user shims file
import yaml
SHIM_FILE = os.path.join(LAB_DIR, 'shims.yaml')
DEFAULT_SHIMS = dict(
    shim_x=0,
    shim_y=0,
    shim_z=0,
    shim_z2=0,
    shim_zx=0,
    shim_zy=0,
    shim_xy=0,
    shim_x2y2=0)
if not os.path.isfile(SHIM_FILE):
    with open(SHIM_FILE, 'w') as f:
        yaml.dump(DEFAULT_SHIMS, f)

# create FID experiment using user shims, frequency, and RF amplitude
fid = FIDApp(
    override_par_files=[
        SHIM_FILE
    ],
    override_pars=dict(
        f=frequency, # use value of frequency from the user input
        a_90=rf_amp, # use rf amplitude from the user input
        t_90=32e-6,
        n_scans=1,
        n_samples=10000,
        t_dw=0.5e-6,
        t_acqdelay=50e-6,
        t_end=0.5
    )
)
fid.main()

Column(sizing_mode='stretch_both')
    [0] Row(sizing_mode='stretch_both')
        [0] Bokeh(Figure)
        [1] Bokeh(Figure)
    [1] Row(sizing_mode='stretch_width')
        [0] Button(align='end', button_type='success', name='Run', sizing_mode='stretch_width')
        [1] Button(align='end', button_type='danger', name='Abort', sizing_mode='stretch_width')
        [2] Progress(height=31, sizing_mode='stretch_width', value=0)
        [3] TextInput(align='end', disabled=True, sizing_mode='stretch_width', value='Idle')

**TO DO: Explain how to identify the NMR signal**

**TO DO: Explain: Rotating reference frames**

Diagrams in here

Tell them what to look for (noise etc) maybe example?

<center><img src="Images/RotatingFrame.png" width="800"></center>

> ---
> #### **Task 2.4: Measure the Offset**
> The spectrum plot on the right side displays the FFT for the FID signal. The x-axis displays the relative frequency of the signal - where 0 Hz is the $f_0$ of the RF pulse. Does your acquired signal peak correspond to your calculated Larmor frequency - why or why not? *we need to reword this explaination i find it confusing*
>
> - Measure the relative frequency offset of the **Spectrum** peak (try the zoom tools at the top right of the plot) and add it to your calculated frequency, then enter the new value into the frequency input and rerun the **FID** tool to check the spectrum peak is now centered. Record this value for your report. *also reword*
> - Open the **Find Frequency** dashboard from the **Dashboard Apps** sidebar and run it. How does the frequency you found compare?
> ---

> ---
> #### **Task 2.5: Retune the Probe**
> - Now that the correct frequency has been measured, go back to **Step Two**, rerun wobble, and retune the probe if necessary.   
> ---


**TO DO: Add new dashboard for this**

## 3. Flip angle

Explain this. Does this relate to the pulse calibration step?
Only the component in the x,y plane is measurable by the probe coil

How flip angle relates to area under pulse

Diagram: 
show spin angles - and the pulse amplitudes diagram

Add the pulse sequence diagram

Pulse calibration showing full rotation


<center><img src="Images/SpinAngleDiagram_2.0.png" width="300"></center>

<center><img src="Images/SpinAngleSignal.png" width="700"></center>

#### **Step Four: Pulse Calibration**

**TO DO: inline dashboard for pulse calibration. Find max by looking at the curve**

BAD :(

With the system tuned, it is time to determine the RF pulse amplitude necessary to achieve a $90^\circ$ tip of the magnetization. Try changing the RF Amplitude above and observing the effect on the initial value of the real part of the **Signal** plot. Roughly what RF Amplitude maximises the signal?

Open the **Pulse Calibration** dashboard from the **Dashboard Apps** sidebar and run it. With **Pulse Calibration** dashboard tool the pulse amplitude for a given pulse length is found to maximize signal - giving the pulse size for a $90^\circ$ tip. Make sure *soft pulse* is **deselected** and the *pulse width* is $32\mathrm{\mu s}$. When the pulse calibration is finished, the results will be automatically saved to the system global variables, and the amplitude value can also be read off the $x$ axis at the position of the dashed line (Note: the axis tick labels use SI prefixes, so 100m means 0.1).

## 4. Decay - T1,T2,T2*
Basic T1, T2. Diagram with these labelled 

Intro homogeneity - maybe hall probe again



Run FID again with correct pulse amplitude. Maximised inital value. Want to maximis the area under the signal - T2*.
Explain the different decays

Diagrams:
- Homogeneity diagram - arrows for different field strengths
- T1 returning to vertical
- T2 dephasing
- Diagram with T2* decay 




<center><img src="Images/T1Decay.png" width="400"></center>

<center><img src="Images/T2StarPhaseDiagram.png" width="1300"></center>

<center><img src="Images/T2StarDecay.png" width="600"></center>

## 5. Correcting B0 Field Homogeneity
revisit title

Once the centre frequency has been established and the system set-up to excite and receive signals of that frequency, the system must be shimmed in order to maximize the homogeneity of the magnetic field. Run the **Autoshim** tool below with the **Coarse** setting to shim the system. The *Coarse* setting will start by resetting all the shims to zero and uses a wide bandwidth, whereas the *Quick* setting loads the saved values and uses a narrow bandwidth; it is suitable for retuning the system once everything is set up. The shims will be saved to `shims.yaml` in your `LAB_DIR`, to be used by the FID experiment above.



> ---
> #### **Task 4.1: Run Autoshim**
> - TO DO 
> ---


In [7]:
# display frequency and rf_amp again as they are used by Autoshim
pn.Row(frequency, rf_amp)

Row
    [0] FloatInput(name='Frequency (Hz)', start=0, step=1000)
    [1] FloatInput(end=1, name='RF Amplitude (..., start=0, value=0.5)

In [8]:
from Autoshim import AutoshimApp # from dashboards-inline directory that was added to sys.path

autoshim = AutoshimApp(
    output_dir=LAB_DIR,
    override_pars=dict(
        f=frequency, # use value of frequency from the user input
        a_90=rf_amp, # use rf amplitude from the user input
        t_90=32e-6,
    )
)
autoshim.main()

Column(sizing_mode='stretch_both')
    [0] Column(sizing_mode='stretch_both')
        [0] Bokeh(Figure)
        [1] Bokeh(Figure)
    [1] Row(sizing_mode='stretch_width')
        [0] Select(height=31, options=['Quick', 'Fine', ...], sizing_mode='stretch_width', value='Coarse')
        [1] Button(align='end', button_type='success', name='Run', sizing_mode='stretch_width')
        [2] Button(align='end', button_type='danger', name='Abort', sizing_mode='stretch_width')
        [3] Progress(height=31, sizing_mode='stretch_width', value=0)
        [4] TextInput(align='end', disabled=True, sizing_mode='stretch_width', value='Idle')

Explain graph

> ---
> #### **Task 4.2: FID Check with Shim**
> - Rerun the FID experiment using the tool below and make note of how the amplitude and decay of the signal has changed compared to without shims in **Step Three**. With a good shim there should still be significant signal at 5 ms. 
> ---


In [9]:
# create FID experiment using user shims, frequency, and RF amplitude
fid2 = FIDApp(
    override_par_files=[
        SHIM_FILE
    ],
    override_pars=dict(
        f=frequency, # use value of frequency from the user input
        a_90=rf_amp, # use rf amplitude from the user input
        t_90=32e-6,
        n_scans=1,
        n_samples=10000,
        t_dw=0.5e-6,
        t_acqdelay=50e-6,
        t_end=0.5
    )
)
fid2.main()

Column(sizing_mode='stretch_both')
    [0] Row(sizing_mode='stretch_both')
        [0] Bokeh(Figure)
        [1] Bokeh(Figure)
    [1] Row(sizing_mode='stretch_width')
        [0] Button(align='end', button_type='success', name='Run', sizing_mode='stretch_width')
        [1] Button(align='end', button_type='danger', name='Abort', sizing_mode='stretch_width')
        [2] Progress(height=31, sizing_mode='stretch_width', value=0)
        [3] TextInput(align='end', disabled=True, sizing_mode='stretch_width', value='Idle')

## 6. Spin Echo 

Diagrams:

Spin echo pulse sequence 

Spin diagrams showing different phase and 180 degree pulse - show realigning



<center><img src="Images/SpinEchoPulse.png" width="800"></center>

<center><img src="Images/SpinEchoPhase.png" width="1300"></center>

## T1 Repetition Time